In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [2]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_stb.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-2-41a7e5b46c0c>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
STB,20210409,22.2,22.9,22.2,22.7,25317200,22.2,22.9,22.2,22.7,500000,1819300,54400,2021-04-09,14,2021,2021-14
STB,20210408,22.5,22.6,22.0,22.2,26665400,22.5,22.6,22.0,22.2,20000,935200,100000,2021-04-08,14,2021,2021-14
STB,20210407,22.3,22.6,22.1,22.3,51625800,22.3,22.6,22.1,22.3,522000,1364900,507600,2021-04-07,14,2021,2021-14


In [3]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [4]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})

fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='STB'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))


In [5]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'STOCK Violin chart')

In [6]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
STB,18.3,2015-04-15,1216740,2015-16,2015
STB,18.5,2015-04-14,590830,2015-16,2015
STB,18.5,2015-04-13,511490,2015-16,2015


In [7]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
STB,22.20,2021-04-08,26665400,2021-14,2021,-2.202643
STB,22.30,2021-04-07,51625800,2021-14,2021,0.450450
STB,22.75,2021-04-06,42374000,2021-14,2021,2.017937


In [8]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [9]:
df['Daily_return'].describe()

count    1499.000000
mean        0.013940
std         2.354023
min        -6.538462
25%        -1.056805
50%         0.000000
75%         1.192253
max        18.918919
Name: Daily_return, dtype: float64

In [10]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [11]:
df['<Close>'].describe()

count    1499.000000
mean       12.521988
std         2.777492
min         7.300000
25%        10.750000
50%        11.750000
75%        13.450000
max        23.150000
Name: <Close>, dtype: float64

In [12]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [13]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                        mode='lines', name='Rolling_volumn'))

In [14]:
df['Volumn'].head(15).describe()

count    15.000000
mean     40.481613
std      20.262868
min      22.973200
25%      25.981450
50%      33.202800
75%      48.894800
max      99.978500
Name: Volumn, dtype: float64

In [15]:
df['Volumn'].head(10).describe()

count    10.000000
mean     47.308400
std      21.825368
min      23.365100
25%      34.150950
50%      44.268900
75%      54.935850
max      99.978500
Name: Volumn, dtype: float64